In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         394 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5.

In [2]:
pip install requests beautifulsoup4 

     |████████████████████████████████| 122kB 6.5MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
# importing libraries
from bs4 import BeautifulSoup
import requests
import urllib.request


In [4]:
# scrap data from wiki


url="https://finkode.com/ap/visakhapatnam.html"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content)

#print(soup.prettify()) # print the parsed data of html

In [5]:
# Extract the table data using BeautifulSoup
table = soup.find('table', attrs={'class':'plist'})
trs = table.find_all('tr')
print("Number of rows in the table(including columns header): ",
      len(trs))

# Extract the text from all the table cells and add all rows
# to a list of rows.
rows = list()
for tr in trs:
    td = tr.find_all('td')
    row = [ele.text.strip() for ele in td]
    if row:
        # Ignore empty rows with no 'td',
        # applicable for the column headers row.
        rows.append(row)

print("Number of rows with data in the table: ", len(rows))

Number of rows in the table(including columns header):  698
Number of rows with data in the table:  697


In [6]:
df = pd.DataFrame(rows, columns=['Neighborhood', 'District', 'pincode'])
print("Dataframe shape: ", df.shape)
df.head()

Dataframe shape:  (697, 3)


,Neighborhood,District,pincode
0,A Koduru S.O,Visakhapatnam,531022
1,A Kothapalle B.O,Visakhapatnam,531022
2,A U Engg College B.O,Visakhapatnam,530003
3,A Veeranarayanam B.O,Visakhapatnam,531027
4,Adakula B.O,Visakhapatnam,531115


In [7]:
# group neighborhoods in the same borough
df = df.groupby(['pincode', 'District'])['Neighborhood'].\
    apply(', '.join).to_frame()
df.reset_index(inplace=True)
df.head()


,pincode,District,Neighborhood
0,530001,Visakhapatnam,"Fortward S.O, Kurupam Market S.O, Visakhapatna..."
1,530002,Visakhapatnam,"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O,..."
2,530003,Visakhapatnam,"A U Engg College B.O, Andhra University S.O, C..."
3,530004,Visakhapatnam,"Gnanapuram S.O, LIC Building S.O, Waltair R S H.O"
4,530005,Visakhapatnam,"Gandhigram S.O (Visakhapatnam), Nausenabagh S...."


In [8]:
df.shape



(90, 3)

In [9]:
# part -2 

#df = pd.read_csv('./bangalore_neighbourhood_original.csv', sep="\t")
import pandas as pd

geospatial_data = pd.read_csv("coordinates.csv")

print("Geospatial dataframe's shape: ", geospatial_data.shape)
geospatial_data.head()

Geospatial dataframe's shape:  (90, 3)


,pincode,Latitude,Longitude
0,530001,17.6998,83.2961
1,530002,17.7121,83.3121
2,530003,17.7243,83.3228
3,530004,17.7230,83.2854
4,530005,17.6657,83.2452


In [10]:
# Concatenate the two dataframes using column "PostalCode" from 'df' and
# i got this error, so i used coverstion "ValueError: You are trying to merge on int64 and object columns"
df['pincode'] = df['pincode'].astype(int)
geospatial_data['pincode'] = geospatial_data['pincode'].astype(int)

df3  = pd.merge(geospatial_data,df, how='inner')

df3.head()

,pincode,Latitude,Longitude,District,Neighborhood
0,530001,17.6998,83.2961,Visakhapatnam,"Fortward S.O, Kurupam Market S.O, Visakhapatna..."
1,530002,17.7121,83.3121,Visakhapatnam,"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O,..."
2,530003,17.7243,83.3228,Visakhapatnam,"A U Engg College B.O, Andhra University S.O, C..."
3,530004,17.7230,83.2854,Visakhapatnam,"Gnanapuram S.O, LIC Building S.O, Waltair R S H.O"
4,530005,17.6657,83.2452,Visakhapatnam,"Gandhigram S.O (Visakhapatnam), Nausenabagh S...."


In [11]:
# create a new test dataframe with reordering the columns
column_names =df3[["pincode", "District", "Neighborhood", "Latitude", "Longitude"]]
column_names.head()

,pincode,District,Neighborhood,Latitude,Longitude
0,530001,Visakhapatnam,"Fortward S.O, Kurupam Market S.O, Visakhapatna...",17.6998,83.2961
1,530002,Visakhapatnam,"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O,...",17.7121,83.3121
2,530003,Visakhapatnam,"A U Engg College B.O, Andhra University S.O, C...",17.7243,83.3228
3,530004,Visakhapatnam,"Gnanapuram S.O, LIC Building S.O, Waltair R S H.O",17.7230,83.2854
4,530005,Visakhapatnam,"Gandhigram S.O (Visakhapatnam), Nausenabagh S....",17.6657,83.2452


In [12]:
column_names.shape

(90, 5)

In [13]:
# part-3 ...........
import json
!pip install geopy 
import geopy



In [14]:
address = 'Visakhapatnam'

geolocator = Nominatim(user_agent="my_trail")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of vishakapatanam are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of vishakapatanam are 17.7231276, 83.3012842.


In [15]:
#creating map of Vishakapatanam
map_Visakhapatnam = folium.Map(
    location=[location.latitude, location.longitude],zoom_start=10)
map_Visakhapatnam

In [16]:
# add markers to map
for lat, lng, district, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['District'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Visakhapatnam)  
    
map_Visakhapatnam

In [17]:
CLIENT_ID = 'my_id' # your Foursquare ID
CLIENT_SECRET = 'my_secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails are ok')

Your credentails are ok


In [18]:
#Let's explore the first neighborhood in our dataframe
df3.loc[0, 'Neighborhood']

neighborhood_latitude = df3.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df3.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df3.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Fortward S.O, Kurupam Market S.O, Visakhapatnam H.O are 17.6998, 83.2961.


In [19]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [20]:
# Get the venues.
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5efdd94bf5be0232e591ee7d'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Vizag',
  'headerFullLocation': 'Vizag',
  'headerLocationGranularity': 'city',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 17.704300004500006,
    'lng': 83.30081478858062},
   'sw': {'lat': 17.695299995499994, 'lng': 83.29138521141937}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '501f2ac7e4b0a0a8012b248a',
       'name': 'Visakhapatnam Harbour',
       'location': {'lat': 17.6965121284197,
        'lng': 83.2987862833524,
        'labeledLatLngs': [{'label': 'display',
          'lat': 17.6965121284197,
          'ln

In [21]:
# Explore the venues
venues = results['response']['groups'][0]['items']
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '501f2ac7e4b0a0a8012b248a',
   'name': 'Visakhapatnam Harbour',
   'location': {'lat': 17.6965121284197,
    'lng': 83.2987862833524,
    'labeledLatLngs': [{'label': 'display',
      'lat': 17.6965121284197,
      'lng': 83.2987862833524}],
    'distance': 463,
    'cc': 'IN',
    'city': 'Vizag',
    'state': 'Andhra Pradesh',
    'country': 'India',
    'formattedAddress': ['Vizag', 'Andhra Pradesh', 'India']},
   'categories': [{'id': '4bf58dd8d48988d1e0941735',
     'name': 'Harbor / Marina',
     'pluralName': 'Harbors / Marinas',
     'shortName': 'Harbor / Marina',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/harbor_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-501f2ac7e4b0a0a8012b248a-0'}]

In [22]:
#Before we proceed, let's borrow the get_category_type function from the Foursquare lab.
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
#Now we are ready to clean the json and structure it into a pandas dataframe

venues = results['response']['groups'][0]['items']

In [24]:
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,lat,lng
0,Visakhapatnam Harbour,Harbor / Marina,17.696512,83.298786


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [26]:
#Let's create a function to repeat the same process to all the neighborhoods in df3_new

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
#code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.
latest_venues = getNearbyVenues(names=df3['Neighborhood'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                                  )


Fortward S.O, Kurupam Market S.O, Visakhapatnam H.O
D.C. Buildings S.O, KGH S.O, Maharanipeta S.O, Turners Choultry S.O
A U Engg College B.O, Andhra University S.O, Chinawaltair S.O, Ghandhi Place S.O, Pithapuram Colony S.O, Vidya Bhavan S.O
Gnanapuram S.O, LIC Building S.O, Waltair R S H.O
Gandhigram S.O (Visakhapatnam), Nausenabagh S.O, Yarada B.O
Industrial Estate S.O (Visakhapatnam), Muralinagar S.O
IRSD Area S.O, Kancharapalem S.O
Marripalem Vuda Colony S.O, N A D S.O, Visakhapatnam Airport S.O
Malkapuram S.O
Autonagar S.O, B H P V S.O, Jaggayyapalem B.O, Mindi B.O, Sheelanagar B.O, Tunglam B.O
P & T Colony (VM) S.O
Naval Base S.O, Naval Dock Yard S.O
Zinc Smelter P S.O
Akkayyapalem S.O, Dwarakanagar S.O, New Colony S.O
Isakathota S.O, L B Colony S.O, M.V.P.Colony S.O
Marripalem S.O
Dabagardens S.O, VM Bus Station S.O
H B Colony S.O
Salagramapuram S.O
Gajuwaka S.O
Gopalapatnam S.O, Narava B.O, NSTL B.O, Prahladapuram B.O, Yellapuvanipalem B.O
Simhachalam Hills B.O, Simhachalam S.O

In [28]:
print(latest_venues.shape)
latest_venues.head()

(85, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Fortward S.O, Kurupam Market S.O, Visakhapatna...",17.6998,83.2961,Visakhapatnam Harbour,17.696512,83.298786,Harbor / Marina
1,"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O,...",17.7121,83.3121,Novotel Visakhapatnam Varun Beach,17.710829,83.316218,Hotel
2,"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O,...",17.7121,83.3121,Novotel Varun Beach,17.710712,83.316263,Restaurant
3,"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O,...",17.7121,83.3121,Inox,17.711077,83.315785,Multiplex
4,"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O,...",17.7121,83.3121,Ming Garden,17.711590,83.316302,Restaurant


In [29]:
#Let's check how many venues were returned for each neighborhood

latest_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"A U Engg College B.O, Andhra University S.O, Chinawaltair S.O, Ghandhi Place S.O, Pithapuram Colony S.O, Vidya Bhavan S.O",3,3,3,3,3,3
"Akkayyapalem S.O, Dwarakanagar S.O, New Colony S.O",8,8,8,8,8,8
"Autonagar S.O, B H P V S.O, Jaggayyapalem B.O, Mindi B.O, Sheelanagar B.O, Tunglam B.O",1,1,1,1,1,1
"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O, Turners Choultry S.O",13,13,13,13,13,13
"Dabagardens S.O, VM Bus Station S.O",9,9,9,9,9,9
"Dayalnagar B.O, Visalakshinagar S.O",1,1,1,1,1,1
"Durganagar B.O, R R V Puram S.O, Venkatapuram B.O",1,1,1,1,1,1
"Duvvada B.O, Pakeertekya B.O, Pedamadaka B.O, Vadlapudi S.O, Vedullanarava B.O",4,4,4,4,4,4
"Fortward S.O, Kurupam Market S.O, Visakhapatnam H.O",1,1,1,1,1,1


In [30]:
#Let's find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(latest_venues['Venue Category'].unique())))

There are 47 uniques categories.


In [31]:
 #Analyze Each Neighborhood


# one hot encoding
manhattan_onehot = pd.get_dummies(latest_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = latest_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()
    

,Neighborhood,ATM,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Boat or Ferry,Bookstore,Brewery,Business Service,Café,Camera Store,Candy Store,Chinese Restaurant,Department Store,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fishing Spot,Food Court,Grocery Store,Gym,Harbor / Marina,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Movie Theater,Multiplex,Optical Shop,Park,Pharmacy,Platform,Playground,Restaurant,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Smoke Shop,Steakhouse,Tennis Court,Track Stadium,Train Station,Vegetarian / Vegan Restaurant
0,"Fortward S.O, Kurupam Market S.O, Visakhapatna...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O,...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [32]:
#And let's examine the new dataframe size.

manhattan_onehot.shape

(85, 48)

In [33]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,ATM,Arts & Crafts Store,Asian Restaurant,Bakery,Bar,Boat or Ferry,Bookstore,Brewery,Business Service,Café,Camera Store,Candy Store,Chinese Restaurant,Department Store,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fishing Spot,Food Court,Grocery Store,Gym,Harbor / Marina,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Movie Theater,Multiplex,Optical Shop,Park,Pharmacy,Platform,Playground,Restaurant,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Smoke Shop,Steakhouse,Tennis Court,Track Stadium,Train Station,Vegetarian / Vegan Restaurant
0,"A U Engg College B.O, Andhra University S.O, C...",0.0,0.00,0.000000,0.00,0.00,0.000000,0.000,0.333333,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.000000,0.0,0.333333,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.333333,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000
1,"Akkayyapalem S.O, Dwarakanagar S.O, New Colony...",0.0,0.00,0.000000,0.00,0.00,0.000000,0.125,0.000000,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.125,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.125000,0.000000,0.250000,0.125000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.125
2,"Autonagar S.O, B H P V S.O, Jaggayyapalem B.O,...",0.0,0.00,0.000000,0.00,0.00,0.000000,0.000,0.000000,1.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000
3,"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O,...",0.0,0.00,0.076923,0.00,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.076923,0.0,0.000000,0.0,0.0,0.153846,0.076923,0.153846,0.000000,0.00,0.000000,0.076923,0.076923,0.0,0.000000,0.0,0.0,0.230769,0.000000,0.0,0.0,0.000000,0.00,0.076923,0.0,0.0,0.0,0.000
4,"Dabagardens S.O, VM Bus Station S.O",0.0,0.00,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.111111,0.0,0.000000,0.0,0.000000,0.0,0.0,0.111111,0.000000,0.111111,0.222222,0.00,0.111111,0.111111,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.222222,0.00,0.000000,0.0,0.0,0.0,0.000
5,"Dayalnagar B.O, Visalakshinagar S.O",0.0,0.00,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,1.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000
6,"Durganagar B.O, R R V Puram S.O, Venkatapuram B.O",0.0,0.00,0.000000,1.00,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000
7,"Duvvada B.O, Pakeertekya B.O, Pedamadaka B.O, ...",0.0,0.00,0.000000,0.25,0.25,0.000000,0.000,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.250000,0.25,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000
8,"Fortward S.O, Kurupam Market S.O, Visakhapatna...",0.0,0.00,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000
9,Gajuwaka S.O,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.200000,0.000000,0.200000,0.000000,0.00,0.200000,0.000000,0.000000,0.0,0.200000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.200000,0.00,0.000000,0.0,0.0,0.0,0.

In [34]:
#Let's confirm the new size

manhattan_grouped.shape

(21, 48)

In [35]:
#Let's print each neighborhood along with the top 5 most common venues


num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A U Engg College B.O, Andhra University S.O, Chinawaltair S.O, Ghandhi Place S.O, Pithapuram Colony S.O, Vidya Bhavan S.O----
            venue  freq
0             Gym  0.33
1         Brewery  0.33
2  Sandwich Place  0.33
3             ATM  0.00
4      Playground  0.00


----Akkayyapalem S.O, Dwarakanagar S.O, New Colony S.O----
                           venue  freq
0              Indian Restaurant  0.25
1  Vegetarian / Vegan Restaurant  0.12
2                      Bookstore  0.12
3            Indie Movie Theater  0.12
4              Electronics Store  0.12


----Autonagar S.O, B H P V S.O, Jaggayyapalem B.O, Mindi B.O, Sheelanagar B.O, Tunglam B.O----
                 venue  freq
0     Business Service   1.0
1                  ATM   0.0
2           Playground   0.0
3    Indian Restaurant   0.0
4  Indie Movie Theater   0.0


----D.C. Buildings S.O, KGH S.O, Maharanipeta S.O, Turners Choultry S.O----
               venue  freq
0         Restaurant  0.23
1  Indian Restaurant  0.15
2

In [36]:
#Let's put that into a pandas dataframe
#First, let's write a function to sort the venues in descending order.


def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"A U Engg College B.O, Andhra University S.O, C...",Gym,Brewery,Sandwich Place,Candy Store,Grocery Store,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market,Electronics Store
1,"Akkayyapalem S.O, Dwarakanagar S.O, New Colony...",Indian Restaurant,Vegetarian / Vegan Restaurant,Hotel,Indie Movie Theater,Bookstore,Electronics Store,Camera Store,Department Store,Food Court,Fishing Spot
2,"Autonagar S.O, B H P V S.O, Jaggayyapalem B.O,...",Business Service,Vegetarian / Vegan Restaurant,Candy Store,Grocery Store,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop
3,"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O,...",Restaurant,Hotel,Indian Restaurant,Multiplex,Asian Restaurant,Steakhouse,Ice Cream Shop,Food Court,Optical Shop,Vegetarian / Vegan Restaurant
4,"Dabagardens S.O, VM Bus Station S.O",Shopping Mall,Indie Movie Theater,Fast Food Restaurant,Hotel,Indian Restaurant,Movie Theater,Multiplex,Vegetarian / Vegan Restaurant,Chinese Restaurant,Farmers Market


In [38]:
#Cluster Neighborhoods
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 3, 2, 2, 4, 2], dtype=int32)

In [39]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

manhattan_merged = df3

# merge Vishakapatanam_grouped with vishakapatanam_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,pincode,Latitude,Longitude,District,Neighborhood,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,530001,17.6998,83.2961,Visakhapatnam,"Fortward S.O, Kurupam Market S.O, Visakhapatna...",4.0,Harbor / Marina,Vegetarian / Vegan Restaurant,Candy Store,Grocery Store,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop
1,530002,17.7121,83.3121,Visakhapatnam,"D.C. Buildings S.O, KGH S.O, Maharanipeta S.O,...",2.0,Restaurant,Hotel,Indian Restaurant,Multiplex,Asian Restaurant,Steakhouse,Ice Cream Shop,Food Court,Optical Shop,Vegetarian / Vegan Restaurant
2,530003,17.7243,83.3228,Visakhapatnam,"A U Engg College B.O, Andhra University S.O, C...",2.0,Gym,Brewery,Sandwich Place,Candy Store,Grocery Store,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market,Electronics Store
3,530004,17.7230,83.2854,Visakhapatnam,"Gnanapuram S.O, LIC Building S.O, Waltair R S H.O",2.0,Platform,Train Station,Arts & Crafts Store,Asian Restaurant,Grocery Store,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market,Electronics Store
4,530005,17.6657,83.2452,Visakhapatnam,"Gandhigram S.O (Visakhapatnam), Nausenabagh S....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
#Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



In [54]:
#Examine Clusters
#Cluster 1


manhattan_merged.loc[manhattan_merged['Cluster_Labels'] == 0, manhattan_merged.columns[[1,2] + list(range(5, manhattan_merged.shape[1]))]]



,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,17.7344,83.2747,0.0,Pharmacy,Indie Movie Theater,Boat or Ferry,Vegetarian / Vegan Restaurant,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop
15,17.7439,83.2519,0.0,ATM,Pharmacy,Harbor / Marina,Grocery Store,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop
24,17.6566,83.1542,0.0,Grocery Store,Farmers Market,Department Store,Park,Pharmacy,Gym,Food Court,Fishing Spot,Fast Food Restaurant,Electronics Store
33,17.8296,83.3896,0.0,Pharmacy,Vegetarian / Vegan Restaurant,Harbor / Marina,Grocery Store,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop


In [67]:
#Cluster 2
manhattan_merged.loc[manhattan_merged['Cluster_Labels'] == 1, manhattan_merged.columns[[1,2] + list(range(5, manhattan_merged.shape[1]))]]

,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,17.8078,83.3549,1.0,Sculpture Garden,Vegetarian / Vegan Restaurant,Candy Store,Grocery Store,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop


In [55]:
#cluster 3
manhattan_merged.loc[manhattan_merged['Cluster_Labels'] == 2, manhattan_merged.columns[[1,2] + list(range(5, manhattan_merged.shape[1]))]]

,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,17.7121,83.3121,2.0,Restaurant,Hotel,Indian Restaurant,Multiplex,Asian Restaurant,Steakhouse,Ice Cream Shop,Food Court,Optical Shop,Vegetarian / Vegan Restaurant
2,17.7243,83.3228,2.0,Gym,Brewery,Sandwich Place,Candy Store,Grocery Store,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market,Electronics Store
3,17.7230,83.2854,2.0,Platform,Train Station,Arts & Crafts Store,Asian Restaurant,Grocery Store,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market,Electronics Store
5,17.7489,83.2667,2.0,Track Stadium,Tennis Court,Bakery,Donut Shop,Department Store,Vegetarian / Vegan Restaurant,Candy Store,Food Court,Fishing Spot,Fast Food Restaurant
8,17.6921,83.2426,2.0,Arts & Crafts Store,Smoke Shop,Bar,Indian Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market
9,17.6992,83.1917,2.0,Business Service,Vegetarian / Vegan Restaurant,Candy Store,Grocery Store,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop
10,17.7416,83.3121,2.0,Ice Cream Shop,Indian Restaurant,Seafood Restaurant,Playground,Chinese Restaurant,Vegetarian / Vegan Restaurant,Candy Store,Fishing Spot,Fast Food Restaurant,Farmers Market
13,17.7294,83.3014,2.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Hotel,Indie Movie Theater,Bookstore,Electronics Store,Camera Store,Department Store,Food Court,Fishing Spot
14,17.7366,83.3389,2.0,Park,Indian Restaurant,Restaurant,Café,Candy Store,Vegetarian / Vegan Restaurant,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market
16,17.7161,83.3001,2.0,Shopping Mall,Indie Movie Theater,Fast Food Restaurant,Hotel,Indian Restaurant,Movie Theater,Multiplex,Vegetarian / Vegan Restaurant,Chinese Restaurant,Farmers Market


In [56]:
#cluster 4

manhattan_merged.loc[manhattan_merged['Cluster_Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Latitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,17.7544,3.0,Fishing Spot,Vegetarian / Vegan Restaurant,Candy Store,Grocery Store,Food Court,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop,Department Store


In [57]:
# cluster 5
manhattan_merged.loc[manhattan_merged['Cluster_Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Latitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,17.6998,4.0,Harbor / Marina,Vegetarian / Vegan Restaurant,Candy Store,Grocery Store,Food Court,Fishing Spot,Fast Food Restaurant,Farmers Market,Electronics Store,Donut Shop
